In [ ]:
import os
import sys
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from neuron import h
h.load_file('stdrun.hoc')

if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from dlutils.cell import Cell, branch_order
from dlutils.utils import extract_mechanisms
from neuroutils.trees import ImpedanceTree
from neuroutils.nodes import ImpedanceNode

In [ ]:
config_file = os.path.join('thorny.json')
config = json.load(open(config_file, 'r'))
optim_dir = config['optimization_folder']
cell_type = config['cell_type']
cell_name = config['cell_name']
optim_run = config['optimization_run']
individual = config['individual']
base_folder = os.path.join(optim_dir, cell_type.upper(), cell_name, optim_run)
swc_file = os.path.join(base_folder, config['swc_file'])
params_file = os.path.join(base_folder, f'individual_{individual}.json')
parameters = json.load(open(params_file, 'r'))
mechanisms = extract_mechanisms(os.path.join(base_folder, 'parameters.json'), cell_name+'_')

In [ ]:
replace_axon = True
add_axon_if_missing = True
cell = Cell('cell_{}'.format(np.random.randint(1000)), swc_file, parameters, mechanisms)
cell.instantiate(replace_axon, add_axon_if_missing, use_dlambda_rule=False,
                 force_passive=True, TTX=False)
soma = cell.morpho.soma[0](0.5)

In [ ]:
F     = 0.         # [Hz]
ω     = 2*np.pi*F  # [rad/s]
if F > 0:
    T = 1000/F     # [ms]

In [ ]:
stim_seg = soma
if F == 0:
    stim = h.IClamp(stim_seg)
    stim.dur = 1e3
else:
    stim = h.Izap(stim_seg)
    stim.f0 = F
    stim.f1 = F
    stim.dur = 200 + 5*T
stim.delay = 500
stim.amp = 0.1

In [ ]:
tree = ImpedanceTree(root_seg=stim_seg)
tree.compute_impedances(F)
tree.compute_attenuations()

In [ ]:
segments = [node.seg for node in tree]
n_segments = len(segments)

In [ ]:
t_rec = h.Vector()
t_rec.record(h._ref_t)
v_rec = []
distances = []
for seg in segments:
    rec = h.Vector()
    rec.record(seg._ref_v)
    v_rec.append(rec)
    coeff = -1 if seg.sec in cell.morpho.dend else 1
    distances.append(coeff * h.distance(soma, seg))
distances = np.array(distances)

In [ ]:
if cell_name == 'DH070813':
    sec = cell.morpho.apic[10]
    n_pts = sec.n3d()
    x = np.array([sec.x3d(i) for i in range(n_pts)])
    y = np.array([sec.y3d(i) for i in range(n_pts)])
    z = np.array([sec.z3d(i) for i in range(n_pts)])
    diam = np.array([sec.diam3d(i) for i in range(n_pts)])
    arc = np.array([sec.arc3d(i) for i in range(n_pts)])
    seg_limits = np.linspace(0, 1, sec.nseg+1)
    seg_x = seg_limits[:-1] + 1/sec.nseg/2
    idx = np.digitize(arc/sec.L, seg_limits) - 1
    idx[idx==sec.nseg] -= 1

    jdx = idx == sec.nseg//2
    middle_mean = np.array([x[jdx],y[jdx],z[jdx]]).mean(axis=1)
    middle_jdx = np.argmin(np.abs(arc/sec.L - 0.5))
    middle = np.array([x[middle_jdx],y[middle_jdx],z[middle_jdx]])
    middle_rec = h.Vector()
    middle_rec.record(sec(0.5)._ref_v)

    jdx = idx == 0
    first_mean = np.array([x[jdx],y[jdx],z[jdx]]).mean(axis=1)
    first_jdx = np.argmin(np.abs(arc/sec.L - seg_x[0]))
    first = np.array([x[first_jdx],y[first_jdx],z[first_jdx]])
    first_rec = h.Vector()
    first_rec.record(sec(seg_x[0])._ref_v)

    dst_points = np.sqrt(np.sum((middle-first)**2))
    dst_means = np.sqrt(np.sum((middle_mean-first_mean)**2))
    dst_neuron = h.distance(sec(seg_x[0]), sec(0.5))

    dst = 0
    for i in range(first_jdx,middle_jdx):
        dst += np.sqrt((x[i]-x[i+1])**2+(y[i]-y[i+1])**2+(z[i]-z[i+1])**2)

    print('Distance computed using middle points: {:.4f} μm.'.format(dst_points))
    print('Distance computed using mean points: {:.4f} μm.'.format(dst_means))
    print('Distance computed by NEURON: {:.4f} μm.'.format(dst_neuron))
    print('Path distance: {:.4f} μm.'.format(dst))

    def LSA_truncated_cone(coords, diams):
        R = np.max(diams)/2
        r = np.max(diams)/2
        h = np.sqrt(np.sum(np.diff(coords,axis=0)**2))
        s = np.sqrt((R-r)**2 + h**2)
        return np.pi * (R+r) * s

    A = 0
    for i in range(n_pts-1):
        coords = np.array([x[i:i+2],y[i:i+2],z[i:i+2]]).T
        A += LSA_truncated_cone(coords, diam[i:i+2])
        
    print('Area computed by NEURON: {:.4f} μm2.'.format(np.sum([seg.area() for seg in sec])))
    print('Area computed using truncated cones: {:.4f} μm2.'.format(A))

In [ ]:
h.tstop = stim.dur + 2*stim.delay
h.v_init = -65
if F==0:
    h.cvode_active(1)
else:
    h.cvode_active(0)
    h.dt = T/1000
h.run()

In [ ]:
t = np.array(t_rec)
V = np.array([np.array(rec) for rec in v_rec])
if F > 0:
    idx = (t>stim.delay+stim.dur-2*T) & (t<stim.delay+stim.dur)
    ΔV = V[:,idx].max(axis=1) - V[:,idx].min(axis=1)
else:
    idx0 = np.where(t<=stim.delay)[0][-1]
    idx1 = np.where(t<=stim.delay+stim.dur)[0][-1]
    ΔV = V[:,idx1] - V[:,idx0]

In [ ]:
if cell_name == 'DH070813':
    V_middle = np.array(middle_rec)
    V_first = np.array(first_rec)

    if F > 0:
        raise NotImplementedError('not implemented yet')
    else:
        ΔV_middle = V_middle[idx1] - V_middle[idx0]
        ΔV_first = V_first[idx1] - V_first[idx0]
    print('Attenuation between points ({:.2f},{:.2f},{:.2f}) and ({:.2f},{:.2f},{:.2f}): {:.5f}.'.\
          format(first[0],first[1],first[2],middle[0],middle[1],middle[2],ΔV_first/ΔV_middle))

    fig,ax = plt.subplots(1, 1, figsize=(5,3))
    ax.plot(t, V_first, 'k', lw=1, label=f'sec({seg_x[0]})')
    ax.plot(t, V_middle, 'r', lw=1, label='sec(0.5)')
    ax.legend(loc='upper left', frameon=False, fontsize=7)
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Vm (mV)')
    sns.despine()
    fig.tight_layout()

In [ ]:
i,j = 0,100
seg_i,seg_j = segments[i], segments[j]
node_i = tree.find_node(ImpedanceNode(seg_i))
node_j = tree.find_node(ImpedanceNode(seg_j))
A_computed = tree.compute_attenuation(seg_j)
A_measured = ΔV[i]/ΔV[j]
print('Attenuation between segments {} and {}: {:.6f} (measured), {:.6f} (computed).'.\
     format(node_i, node_j, A_measured, A_computed))

In [ ]:
A_k_measured = ΔV[0] / ΔV[1:]
A_k_computed = [tree.compute_attenuation(seg_k) for seg_k in segments[1:]]

In [ ]:
A = pickle.load(open(f'A_{cell_type}.pkl','rb'))
D = pickle.load(open(f'D_{cell_type}.pkl','rb'))

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(8,4), width_ratios=(1,3))

lim = [A_k_measured.min()*0.999, A_k_measured.max()*1.001]
ax[0].plot(lim, lim, lw=2, color=[.4,.4,.4])
ax[0].plot(A_k_measured, A_k_computed, 'ko', markerfacecolor='w', markersize=4)
ax[0].grid(which='major', axis='both', ls=':', lw=0.5, color=[.6,.6,.6])
ax[0].set_xlabel('Measured attenuation')
ax[0].set_ylabel('Computed attenuation')

ax[1].plot(distances[1:], np.log(A_k_measured), 'k.', markersize=3)
ax[1].plot(distances[1:], np.log(A_k_computed), 'r.', markersize=3)
for k in A:
    ax[1].plot(D[k], np.log(A[k]), 'm.', ms=1)
ax[1].set_xlabel('Distance from soma (μm)')

sns.despine()
fig.tight_layout()